In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.preprocessing import PolynomialFeatures

pio.templates.default = "plotly_dark"

In [2]:
# Leitura do dataset
DATASET_PATH = '../data/marketing_sales_data.csv'
data = pd.read_csv(DATASET_PATH, usecols=['Radio', 'Social Media', 'Sales', 'TV'])

# Renomeando colunas 
data = data.rename({'Radio': 'radio', 'Social Media': 'social_media', 'Sales': 'sales', 'TV': 'tv'}, axis=1)
data.head()

,tv,radio,social_media,sales
0,Low,3.518070,2.293790,55.261284
1,Low,7.756876,2.572287,67.574904
2,High,20.348988,1.227180,272.250108
3,Medium,20.108487,2.728374,195.102176
4,High,31.653200,7.776978,273.960377


In [3]:
data.shape

(572, 4)

In [4]:
fig = px.scatter(data, x='radio', y='sales')
fig

In [5]:
# Variáveis: X -> preditora e Y -> resposta
X = ['radio']
Y = 'sales'

# Ajustando o modelo
model = LinearRegression()
model.fit(data[X], data[Y])

# Salvando os coeficientes
b0 = model.intercept_
b1 = model.coef_[0]

print(b0, b1)

43.87894296441647 8.299820328968238


In [6]:
fig.add_trace(go.Scatter(x=data.radio, y=b0 + data.radio*b1, mode="lines", name="Regressão Linear"))

In [8]:
data['predicted'] = (b0 + data.radio*b1)
data['residuals'] = data.sales - data.predicted
data.head()

,tv,radio,social_media,sales,residuals,predicted
0,Low,3.518070,2.293790,55.261284,-17.817004,73.078289
1,Low,7.756876,2.572287,67.574904,-40.684714,108.259618
2,High,20.348988,1.227180,272.250108,59.478221,212.771887
3,Medium,20.108487,2.728374,195.102176,-15.673594,210.775770
4,High,31.653200,7.776978,273.960377,-32.634439,306.594817


In [9]:
residuals_fig = px.scatter(data, x='predicted', y='residuals')
residuals_fig

In [12]:
poly = PolynomialFeatures(degree=4, include_bias=False)
poly_features = poly.fit_transform(data[X])

poly_reg_model = LinearRegression()
poly_reg_model.fit(poly_features, data[Y])

# Salvando os coeficientes
b0 = poly_reg_model.intercept_
b1 = poly_reg_model.coef_[0]
b2 = poly_reg_model.coef_[1]
b3 = poly_reg_model.coef_[2]
b4 = poly_reg_model.coef_[3]

print(b0, b1, b2, b3, b4)

63.30252227344636 2.6217608211876127 0.3634885151451695 -0.005734623299280485 -1.9704750554572215e-05


In [13]:
fig = px.scatter(data, x='radio', y='sales')
fig.add_trace(go.Scatter(x=data.radio, y=b0+(data.radio*b1)+(data.radio*data.radio*b2)+(data.radio*data.radio*data.radio*b3), mode="lines", name="Regressão 3° grau"))

In [59]:
data2 = pd.DataFrame({'x':data.radio, 'y': b0+(data.radio*b1)+(data.radio*data.radio*b2)+(data.radio*data.radio*data.radio*b3) + (data.radio*data.radio*data.radio*data.radio*b4)})

In [60]:
data2.sort_values(by='x', inplace=True)
data2.head()

,x,y
342,0.109106,63.592890
76,0.132418,63.656051
208,0.216101,63.886005
346,0.388040,64.374266
493,0.426664,64.486858


In [61]:
# fig = px.scatter(data, x='radio', y='sales')
fig.add_trace(go.Scatter(x=data2.x, y=data2.y, mode="lines", name="Regressão 3° grau"))